In [289]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

#Убираем FutureWarning для лучшей читаемости:) 
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Загрузка датасета

In [290]:
note = '''Примечание:\n
1. Gender - пол. 0 - женский, 1 - мужской\n
2. Age - возраст.\n
3. Chronotype - хронотип. 0 - сова, 1 - жаворонок\n
4. Wake_up - время подъёма (в виде целого числа или числа с плавающей точкой).\n
5. Far_from_MAI? - далеко ли проживает от МАИ? 0 - нет, 1 - да\n
6. Personality_type - тип личности. 0 - экстраверт, 1 - интроверт\n
7. Work - наличие работы. 0 - нет, 1 - есть.\n
8. Salary_satisfaction - удовлетворённость зарплатой. 0 - нет, 1 - да.\n
9. TARGET_CLASS - целевой показатель (присутствие на паре 07.09). 0 - не присутствовал, 1 - присутствовал.'''

In [291]:
raw_data = pd.read_csv('Math4DS_Lk1/knn_dataset.csv', sep = ';')
raw_data

,Gender,Age,Chronotope,Wake_up,Far_from_MAI?,Personality_type,Work,Salary_satisfaction,TARGET_CLASS
0,0,22,0,8.0,1,0,1,1,1
1,1,21,0,9.0,0,0,1,0,0
2,1,22,0,10.0,1,0,0,0,1
3,1,22,0,10.0,1,1,1,1,0
4,0,21,0,11.0,1,0,1,1,1
5,1,21,0,10.0,0,0,1,1,0
6,1,22,0,6.0,1,1,1,0,0
7,1,22,0,9.0,1,0,0,0,1
8,1,22,0,11.0,1,0,1,1,0
9,1,21,1,7.5,1,1,1,0,0


## Стандартизация датасета

In [292]:
scaler = StandardScaler()
scaler.fit(raw_data.drop('TARGET_CLASS', axis=1))  # Обучения scaler на нашем датасете
scaled_features = scaler.transform(raw_data.drop('TARGET_CLASS', axis=1))  # Стандартизируем признаки
scaled_data = pd.DataFrame(scaled_features, columns = raw_data.drop('TARGET_CLASS', axis=1).columns)  # Делаем из массива датафрейм
scaled_data

,Gender,Age,Chronotope,Wake_up,Far_from_MAI?,Personality_type,Work,Salary_satisfaction
0,-2.121320,0.755929,-0.316228,-0.747812,0.471405,-0.755929,0.471405,0.912871
1,0.471405,-1.322876,-0.316228,-0.062318,-2.121320,-0.755929,0.471405,-1.095445
2,0.471405,0.755929,-0.316228,0.623177,0.471405,-0.755929,-2.121320,-1.095445
3,0.471405,0.755929,-0.316228,0.623177,0.471405,1.322876,0.471405,0.912871
4,-2.121320,-1.322876,-0.316228,1.308672,0.471405,-0.755929,0.471405,0.912871
5,0.471405,-1.322876,-0.316228,0.623177,-2.121320,-0.755929,0.471405,0.912871
6,0.471405,0.755929,-0.316228,-2.118802,0.471405,1.322876,0.471405,-1.095445
7,0.471405,0.755929,-0.316228,-0.062318,0.471405,-0.755929,-2.121320,-1.095445
8,0.471405,0.755929,-0.316228,1.308672,0.471405,-0.755929,0.471405,0.912871
9,0.471405,-1.322876,3.162278,-1.090560,0.471405,1.322876,0.471405,-1.095445


## Разделение датасета на обучающие и тестовые данные

In [293]:
x = scaled_data
y = raw_data['TARGET_CLASS']
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.33)

## Реализация модели k-ближайших соседей с помощью библиотек scikit-learn

In [294]:
# Cоздаём модель k-ближайших соседей
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(x_training_data, y_training_data)

# Делаем предсказание
predictions = model.predict(x_test_data)

# Оценка точности
print(classification_report(y_test_data, predictions, zero_division=True))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4



## Реалзиация модели k-ближайших соседей с помощью написания алгоритма вручную

In [295]:
#Евклидово расстояние
def euclidean_distance(data1, data2):
    distance = 0
    for i in range (len(data1) - 1):
        distance += (data1[i] - data2[i]) ** 2
    return math.sqrt(distance)


# Получаем соседей
def get_neighbors(train, test, k=1):
    # Массив из пар значений: (метка класса соседа, расстояние до соседа)
    distances = [(train[i][-1], euclidean_distance(train[i], test))
                  for i in range (len(train))] 
    distances.sort(key=lambda elem: elem[1])  # Сортируем по расстоянию
    neighbors = [distances[i][0] for i in range (k)]  # Массив меток классов k ближайших соседей
    return neighbors


#Считаем, сколько объектов каждого класса присутствует среди k ближайших соседей,
#а затем причислим целевой объект к тому классу, объектов которого больше всего.
def prediction(neighbors):
    count = {}
    for instance in neighbors:
        if instance in count:
            count[instance] +=1
        else :
            count[instance] = 1
    target = max(count.items(), key=lambda x: x[1])[0]  # Определяем класс целевого объекта (каких объектов класса больше, тот и берём)
    return target


# Определение точности предсказания (отношение верных прогнозов к общему количеству прогнозов)
def accuracy(test, test_prediction):
    correct = 0
    for i in range (len(test)):
        if test[i][-1] == test_prediction[i]:
            correct += 1
    return (correct / len(test))

In [296]:
#Для простоты реализации алгоритма объединим датафреймы с признаками объектов и метками их классов, а затем преобразуем их в numpy-массивы
training_data_concat = pd.concat([x_training_data,y_training_data], axis = 1).to_numpy()
test_data_concat = pd.concat([x_test_data,y_test_data], axis = 1).to_numpy()

In [297]:
# Делаем предсказание
predictions = []
for i in range (len(test_data_concat)):
    neighbors = get_neighbors(training_data_concat, test_data_concat[i], k=1)
    result = prediction(neighbors)
    predictions.append(result)
accuracy = accuracy(test_data_concat, predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 0.75


## Проверка работы модели на конкретном человеке

In [301]:
data_instance = []  # Массив с экземпляром данных
print(note, end='\n')
for i in list(raw_data.drop(columns='TARGET_CLASS')):
    n=input(f'Введите число cогласно примечанию: {i}')
    data_instance.append(n)

Примечание:

1. Gender - пол. 0 - женский, 1 - мужской

2. Age - возраст.

3. Chronotype - хронотип. 0 - сова, 1 - жаворонок

4. Wake_up - время подъёма (в виде целого числа или числа с плавающей точкой).

5. Far_from_MAI? - далеко ли проживает от МАИ? 0 - нет, 1 - да

6. Personality_type - тип личности. 0 - экстраверт, 1 - интроверт

7. Work - наличие работы. 0 - нет, 1 - есть.

8. Salary_satisfaction - удовлетворённость зарплатой. 0 - нет, 1 - да.

9. TARGET_CLASS - целевой показатель (присутствие на паре 07.09). 0 - не присутствовал, 1 - присутствовал.


Введите число cогласно примечанию: Gender 1
Введите число cогласно примечанию: Age 20
Введите число cогласно примечанию: Chronotope 0
Введите число cогласно примечанию: Wake_up 8.5
Введите число cогласно примечанию: Far_from_MAI? 1
Введите число cогласно примечанию: Personality_type 0
Введите число cогласно примечанию: Work 1
Введите число cогласно примечанию: Salary_satisfaction 1


In [302]:
df_instance = pd.DataFrame(data = [data_instance], columns = list(raw_data.drop(columns = 'TARGET_CLASS')))  # Преобразуем массив тестовых данных в датафрейм
scaled_data_instance = scaler.transform(df_instance)  # Стандартизируем признаки

In [303]:
neighbors = get_neighbors(training_data_concat, scaled_data_instance[0], k=1)
result = prediction(neighbors)
print('Пришёл' if result else 'Не пришёл')

Пришёл


## Подбор параметра k

In [304]:
error_rates = []

for i in np.arange(1, 5):
    model = KNeighborsClassifier(n_neighbors = i)
    model.fit(x_training_data, y_training_data)
    new_predictions = model.predict(x_test_data)
    error_rates.append(np.mean(new_predictions != y_test_data))

print(f'Минимальное отклонение: {np.array(error_rates).min()*100}%, лучшее значение k: {np.array(error_rates).argmin()+1}')

Минимальное отклонение: 25.0%, лучшее значение k: 1
